<a href="https://colab.research.google.com/github/Guillemolina01/PEC1-futbol/blob/main/Estad%C3%ADsticas_de_futbolistas_de_las_3_grandes_ligas_del_22_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PRÁCTICA 1 - Estadísticas de futbolistas de las 3 grandes ligas del 22-23 (La Liga, Bundesliga, Premier League)**

---



1. Importamos Librerías y Módulos


*   requests: Para hacer solicitudes HTTP.
*   json: Para manejar datos en formato JSON.
*   pandas as pd: Para trabajar con datos en formato de DataFrame.
*   random: Para generar números aleatorios.
*   time: Para manejar el tiempo y pausar ejecuciones.
*   BeautifulSoup: Para hacer scraping web y extraer información de páginas HTML.
*   from google.colab import files: Para trabajar con archivos en Google Colab, como subir y descargar archivos.










In [ ]:
import requests
import json
import pandas as pd
import random
import time
from bs4 import BeautifulSoup
from google.colab import files

2. Accedemos a la página web 'understat.com' para obtener datos de la liga 'La Liga' de la temporada 2022.

In [ ]:
# Definimos la URL base del sitio web
url = 'https://understat.com'
# Lista de ligas que queremos analizar
leagues = ['La_liga', 'EPL','Bundesliga']
# Lista de temporadas que nos interesan
seasons = ['2022']
# Realizamos una solicitud HTTP para obtener la página de la liga 'La Liga' de la temporada 2022
response = requests.get(url+'/'+leagues[0]+'/'+seasons[0])
# Extraemos el User-Agent de la solicitud realizada
user_agent = response.request.headers.get('User-Agent')

3.
En esta parte del código, hemos definido una función llamada get_page que realiza solicitudes HTTP a una URL dada. Esta función utiliza un User-Agent específico para evitar ser bloqueado y también introduce un retraso aleatorio entre 1 y 5 segundos antes de procesar la respuesta. Si la solicitud es exitosa (código de estado 200), la función imprime el contenido HTML de la página de manera legible gracias a la librería BeautifulSoup. Si la solicitud no es exitosa, imprime un mensaje indicando que el contenido no se encontró.

In [ ]:
# Función para hacer una solicitud HTTP con user agent y retraso aleatorio con output legible
def get_page(url):
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        time.sleep(random.uniform(1, 5))
        soup = BeautifulSoup(response.text, 'html.parser')
        print(soup.prettify())  # Imprimir el contenido de la página
    else:
        print("solicitud no exitosa. contenido no encontrado.")

4. Aquí tenemos una función llamada players_data que recibe como argumento el nombre de una liga. Esta función se encarga de obtener y procesar datos de jugadores de esa liga para la temporada 2022 desde el sitio web 'understat.com'.

Primero, construye la URL específica para la liga y temporada dadas. Luego, utiliza la función get_page para obtener el contenido de la página.

Después, busca los scripts dentro del contenido de la página y localiza el que contiene los datos de los jugadores. Extrae el fragmento de datos JSON de este script y lo decodifica para convertirlo en un diccionario de Python.

Finalmente, crea un DataFrame de Pandas con estos datos y lo devuelve como resultado de la función.

In [ ]:
def players_data(league):
    # URL base y temporada
    url = 'https://understat.com/league'
    season = '2022'

    # Estructura de la URL para la liga específica
    league_url = f'{url}/{league}/{season}'

    # Realizamos la solicitud HTTP y obtenemos el contenido de la página
    response = get_page(league_url)

    # Encontramos los scripts en la página
    scripts = response.find_all('script')

    # Iteramos sobre los scripts para encontrar el que contiene los datos de los jugadores
    string_with_json_obj = ''
    for el in scripts:
        if 'playersData' in str(el):
            string_with_json_obj = str(el).strip()

    # Extraemos los datos JSON del script
    ind_start = string_with_json_obj.index("('") + 2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]

    # Decodificamos la cadena JSON
    json_data = json_data.encode('utf8').decode('unicode_escape')

    # Convertimos los datos JSON en un diccionario de Python
    data_list = json.loads(json_data)

    # Creamos un DataFrame a partir del diccionario
    df = pd.DataFrame(data_list)

    return df

In [ ]:
get_page(url)

5. Estas líneas de código llaman a la función players_data con el argumento 'La_liga', que corresponde a la liga 'La Liga'. La función retorna un DataFrame de Pandas con los datos de los jugadores de La Liga para la temporada 2022, y este DataFrame se asigna a la variable df_laliga.

Luego, df_laliga.head() muestra las primeras cinco filas del DataFrame para que podamos echar un vistazo inicial a los datos.

Repetimos esto para cada una de las tres ligas que incluimos

In [ ]:
df_laliga = players_data('La_liga')
df_laliga.head()

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup
0,227,Robert Lewandowski,34,2862,23,25.93928100913763,7,6.353777157142758,135,36,2,1,F S,Barcelona,23,25.196005009114742,36.746243976056576,9.871428990736604
1,2370,Karim Benzema,24,2056,19,23.95281184464693,3,6.247546784579754,107,48,1,0,F,Real Madrid,12,18.006582662463188,25.88561362028122,8.579489946365356
2,866,Joselu,34,2989,16,15.871623763814569,2,2.09515243396163,93,22,2,0,F S,Espanyol,11,12.15523842163384,13.920117577537894,2.132707592099905
3,2270,Antoine Griezmann,38,2853,15,11.751190435141325,16,15.643586406484246,111,85,2,0,F M S,Atletico Madrid,15,11.751190435141325,30.893705546855927,11.25261721946299
4,2543,Borja Iglesias,35,2406,15,15.634133275598288,3,2.9218252673745155,63,22,2,1,F S,Real Betis,10,11.174591269344091,15.961690902709961,3.046393381431699


In [ ]:
df_epl = players_data('EPL')
df_epl.head()

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup
0,8260,Erling Haaland,35,2803,36,32.761399537324905,8,5.8491098545491695,123,29,5,0,F S,Manchester City,29,27.433209866285324,31.963398084044456,3.285816218703985
1,647,Harry Kane,38,3414,30,23.06444012373686,3,7.547407771460712,133,57,6,0,F,Tottenham,25,18.497427217662334,25.079117983579636,5.095352162607014
2,998,Ivan Toney,33,2960,20,21.69119757413864,4,4.909892840310931,94,26,9,0,F,Brentford,14,16.363018790259957,18.1100468672812,4.024585669860244
3,1250,Mohamed Salah,38,3307,19,23.340063713490963,12,8.720245610922575,125,65,2,0,F M S,Liverpool,17,20.295387655496597,32.54413793236017,8.866782674565911
4,468,Callum Wilson,31,1911,18,18.856768030673265,5,3.567963434383273,73,24,4,0,F S,Newcastle United,15,16.57326026633382,15.517562381923199,1.810890594497323


In [ ]:
df_bundes = players_data('Bundesliga')
df_bundes.head()

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup
0,3300,Christopher Nkunku,25,1897,16,16.290899142622948,4,4.9029772616922855,74,38,4,0,F M S,RasenBallsport Leipzig,13,13.25986534357071,19.322517298161983,4.799339268356562
1,6098,Niclas Füllkrug,28,2482,16,13.430156659334898,5,3.7504996675997972,80,33,4,0,F,Werder Bremen,11,9.641273576766253,15.179124631918967,4.494084726087749
2,5266,Vincenzo Grifo,33,2461,15,11.618569439277053,5,6.470908637158573,66,55,2,0,F M S,Freiburg,8,5.556354703381658,12.240157168358564,6.598483877256513
3,5743,Randal Kolo Muani,32,2671,15,14.64113382063806,11,7.074136344715953,77,41,4,1,F M S,Eintracht Frankfurt,13,13.125744855031371,21.198371678590775,3.378522824496031
4,993,Serge Gnabry,34,1950,14,11.549345068633556,5,5.648855604231358,82,36,0,0,F M S,Bayern Munich,13,10.791569046676159,24.274701070040464,10.335147313773632



6. En este fragmento de código, estamos combinando los DataFrames df_bundes, df_epl, y df_laliga en un único DataFrame merged_df utilizando la función pd.concat() de Pandas. Estamos configurando ignore_index=False para mantener los índices originales de cada DataFrame.

Luego, imprimimos el DataFrame combinado merged_df para visualizar todos los datos juntos en un único DataFrame.

In [ ]:
# Combinamos los DataFrames de las ligas Bundesliga, EPL y La Liga en un único DataFrame
merged_df = pd.concat([df_bundes, df_epl, df_laliga], ignore_index=False)

# Mostramos el DataFrame resultante
print(merged_df)

        id              player_name games  time goals                  xG  \
0     3300       Christopher Nkunku    25  1897    16  16.290899142622948   
1     6098          Niclas Füllkrug    28  2482    16  13.430156659334898   
2     5266           Vincenzo Grifo    33  2461    15  11.618569439277053   
3     5743        Randal Kolo Muani    32  2671    15   14.64113382063806   
4      993             Serge Gnabry    34  1950    14  11.549345068633556   
..     ...                      ...   ...   ...   ...                 ...   
579  11524  José Antonio de la Rosa     2    40     0                   0   
580  11525       Babatunde Akinsola     1     5     0                   0   
581  11527             Lamine Yamal     1     2     0  0.4306701719760895   
582  11591          Francisco Mwepu     1    17     0                   0   
583  11620        Rubén Quintanilla     1     1     0                   0   

    assists                    xA shots key_passes yellow_cards red_cards  


7. En estas últimas líneas de código, estamos guardando el DataFrame combinado merged_df en un archivo CSV llamado 'datos__jugadores_futbol_2022_laligabundesligapremierleague.csv'. Utilizamos el método to_csv() de Pandas para esto, y configuramos index=False para que no se incluyan los índices en el archivo CSV.

Luego, con files.download(), descargamos el archivo CSV recién creado desde el entorno de Google Colab.

In [ ]:
# Guardamos el DataFrame resultante en un archivo CSV
merged_df.to_csv('datos__jugadores_futbol_2022_laligabundesligapremierleague.csv', index=False)

# Descargamos el archivo CSV
files.download('datos__jugadores_futbol_2022_laligabundesligapremierleague.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip3 freeze > requirements.txt